# MILP for COPP

In [182]:
# import modules
from gurobipy import *
import numpy as np
import pandas as pd
import pyodbc
import re

In [183]:
# connect to the database
conn_str = 'DSN=JDATA;database=WarehouseModel;Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [184]:
# get data from database
lineItem = pd.read_sql("SELECT ItemID FROM LineItem", conn)
pickers = pd.read_sql("SELECT PickerID, Speed, Capacity, PlaceTime FROM Pickers", conn)
transporters = pd.read_sql("SELECT TransporterID, Speed, Capacity, UnloadTime FROM Transporters", conn)
Sku_L = pd.read_sql("SELECT LineItem.ItemID, Skus.PickNodeID FROM LineItem INNER JOIN Skus ON LineItem.SkuID = Skus.SkuID", conn)
Nodes = pd.read_sql("SELECT ID, Node FROM Nodes WHERE NOT Type='MyTransferNode' AND NOT Type='MyDepotNode'", conn)
# dataframe and list of all physic nodes (Home, Picking locations, Handoff spots, Depot)
PhyNodes = pd.read_sql("SELECT Node FROM Nodes WHERE NOT Type='MyTransferNode'", conn)
PhyNodes_list = PhyNodes['Node'].tolist()
# get some setting paras
# number of items
NumItems = len(lineItem)
# number of depot replicas
DReps = 10
# depots dataframe
Depots = pd.DataFrame(list(range(1,DReps+1)), columns = ['Reps'])
# get travel time matrix from TimeMatrix.xlsx
Time_matrix = pd.read_excel('TimeMatrix.xlsx', sheet_name = 'TimeMatrix', header=None).fillna(0)

## Gurobi Model

In [185]:
# create empty model object
model = Model('COPP')
# set time limit for optimization
model.setParam('TimeLimit', 60)

Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


### Parameters and keys

In [186]:
# define the speed, capacity, place time for each picker
_, Vp, Cp, PLT = multidict(pickers.set_index(['PickerID'])[['Speed','Capacity','PlaceTime']].T.to_dict('list'))
# define the speed, capacity, unload time for each transporter
_, Vt, Ct, ULT = multidict(transporters.set_index(['TransporterID'])[['Speed','Capacity','UnloadTime']].T.to_dict('list'))

# Some useful dataframes
P = pickers[['PickerID']]  # pickers
T = transporters[['TransporterID']] # transporters
I = lineItem  # items
HR = Nodes.drop(0) # all physical handoff spots
# all picking locations
RI = pd.DataFrame(Sku_L.apply(lambda x: 'MyPickNode'+str(x['PickNodeID'])+'_PL'+str(x['ItemID']), axis = 1), columns=['Node'])
HRI = pd.DataFrame() # all handoff replicas
#complete HRI df
for i in range(NumItems):
    HRi = HR.copy()
    HRi['Node'] = HRi['Node'] + '_I'+str(i+1)
    HRI = HRI.append(HRi,ignore_index=True)
# depot replicas
D = pd.DataFrame(Depots.apply(lambda x: 'DepotRep_'+ str(x['Reps']), axis =1), columns = ['Reps'])

## Decision Variables

### Makespan decision variable -- m

In [187]:
m =  model.addVar(vtype=GRB.CONTINUOUS, name = 'm')

### Assignment decision variables -- a[e1, e2, i]

In [188]:
# define the set of all items, pickers and transporters. For decision variable a[e1,e2,i]
# e1: picker
# e2: transporter
# i: item
I_P = pd.merge(I, P, how='cross')
I_P_T = pd.merge(I_P , T, how='cross')
I_P_T['Cell'] = 1
# create dicts for decision variable
Da, _ = multidict(I_P_T.set_index(['PickerID', 'TransporterID', 'ItemID']).to_dict()['Cell'])
# create the decision variable
a = model.addVars(Da, vtype=GRB.BINARY, name = 'A')

### Picker item sequence decision variables -- X[e1, i, j]

In [189]:
# define the set of all pickers and item picking sequence segments. For decision variabel X[e1,i,j]
# e1: picker
# i: picking item (item 0 is the virtual start item)
# l: next picking item (item -1 is the virtual end item)
I_I = pd.merge(I, I, how='cross')
I_I['Keep']=I_I.apply(lambda x: False if x['ItemID_x'] == x['ItemID_y'] else True, axis = 1)
I_I = I_I[I_I['Keep']].reset_index(drop=True)
# add virtual items sequences
for i in range(1,NumItems+1):
    I_I.loc[len(I_I.index)] = [0, i, True]
for i in range(1,NumItems+1):
    I_I.loc[len(I_I.index)] = [i,-1, True]    
P_I_I = pd.merge(P,I_I, how='cross')
P_I_I.columns = ['e1','i','j','Keep']

# create dicts for decision variable
DX, _ = multidict(P_I_I.set_index(['e1', 'i', 'j']).to_dict()['Keep'])
# create the decision variable
X = model.addVars(DX, vtype=GRB.BINARY, name = 'X')

### Picker node sequence decision variables -- Y[e1, k ,k1]

In [190]:
# define the sets of all pickers and travel segements. For decision Variable Y[e1,k,k1]
# e1: picker
# k: previous node
# k1: next node (Picking location, Handoff spots replicas, Home node replicas)

# set when k1 is handoff replicas
# complete P_RI_HRI df
# HR: the set of physical handoff spots
RI_HRI = pd.merge(Sku_L,HR, how='cross')
RI_HRI['k'] = RI_HRI.apply(lambda x: 'MyPickNode'+str(x['PickNodeID'])+'_PL'+str(x['ItemID']), axis = 1)
RI_HRI['k1'] = RI_HRI.apply(lambda x: x['Node']+'_I'+str(x['ItemID']), axis = 1)
P_RI_HRI = pd.merge(P,RI_HRI, how='cross')[['PickerID','k','k1']]
P_RI_HRI.columns = ['e1','k','k1']

# set when k1 is Picking node
# HRI: the set of all handoff spots replicas
# RI: the set of all picking locations
HRI_RI = pd.merge(Sku_L,HRI, how='cross')
HRI_RI['k1'] = HRI_RI.apply(lambda x: 'MyPickNode'+str(x['PickNodeID'])+'_PL'+str(x['ItemID']), axis = 1)
droplist = []
for i in range(len(HRI_RI)):
    if re.findall("\d+",HRI_RI.at[i,'Node'])[-1] == str(HRI_RI.at[i,'ItemID']):
        droplist.append(i)
HRI_RI = HRI_RI.drop(droplist)
P_HRI_RI = pd.merge(P,HRI_RI, how='cross')[['PickerID','Node','k1']]
P_HRI_RI.columns = ['e1','k','k1']
Hp = P.copy()
Hp.columns = ['e1']
Hp['k'] = Hp.apply(lambda x: 'MyHome_P'+str(x['e1']), axis =1)
P_H_RI = pd.merge(Hp,RI,how='cross')
P_H_RI.columns = ['e1','k','k1']
P_HHRI_RI = pd.concat([P_HRI_RI, P_H_RI], ignore_index=True)

# set when k1 is home node replicas
P_HRI_H = pd.merge(Hp, RI_HRI['k1'].to_frame('k'), how='cross')[['e1','k_y','k_x']]
P_HRI_H.columns = ['e1','k','k1']

# get the final sets and dict
Y_P = pd.concat([P_RI_HRI, P_HHRI_RI, P_HRI_H], ignore_index=True)
Y_P['Cell'] = 1
DYp, _ = multidict(Y_P.set_index(['e1', 'k', 'k1']).to_dict()['Cell'])

# create the decision variable
Yp = model.addVars(DYp, vtype=GRB.BINARY, name = 'Yp')

In [191]:
# define the function that set the time value for each node sequence
def setTimeforTLs_P(row):
    head_node = row['k'][:row['k'].index('_')]
    tail_node = row['k1'][:row['k1'].index('_')]
    head_id = PhyNodes_list.index(head_node)
    tail_id = PhyNodes_list.index(tail_node)
    return(Time_matrix.iloc[head_id, tail_id])

Y_P['Time'] = Y_P.apply(lambda x: setTimeforTLs_P(x), axis = 1)
# define travel time for each possible node sequence for picker
_, Lp = multidict(Y_P.set_index(['k', 'k1']).to_dict()['Time'])

### Transporter node sequence decision variables -- Y[e2, k ,k1]

In [192]:
# define a function that is used to decide wheteher a node seuqnece (HRI_HRI) is feasible
def T_NodeSeqFeasibility(row):
    head_item = row['Node_x'][row['Node_x'].index('_'):]
    tail_item = row['Node_y'][row['Node_y'].index('_'):]
    if head_item == tail_item:
        return (False)
    else:
        return(True)

In [193]:
# define the sets of all transporters and travel segements. For decision Variable Y[e2,k,k1]
# e2: transporter
# k: previous node
# k1: next node (Depot replicas, Handoff spots replicas, Home node replicas)

# set when k1 is depot replicas
HRI_D = pd.merge(HRI, D, how='cross')
T_HRI_D = pd.merge(T, HRI_D[['Node','Reps']], how='cross')
T_HRI_D.columns = ['e2','k','k1']

# # set when k1 is home node replicas
Ht = T.copy()
Ht.columns = ['e2']
Ht['k1'] = Ht.apply(lambda x: 'MyHome_T'+str(x['e2']), axis =1)
D_H = pd.merge(D, Ht, how='cross')
T_D_H = D_H[['e2','Reps','k1']]
T_D_H.columns = ['e2','k','k1']

# set when k1 is handoff spots replicas
# when k is home node
H_HRI = pd.merge(Ht, HRI, how='cross')
T_H_HRI = H_HRI[['e2','k1','Node']]
T_H_HRI.columns = ['e2','k','k1']
# when k is depot replicas
D_HRI = pd.merge(D, HRI, how='cross')
# D_HRI['k'] = D_HRI.apply(lambda x: 'DepotRep_'+ str(x['Reps']), axis =1)
T_D_HRI = pd.merge(T, D_HRI, how='cross')[['TransporterID','Reps','Node']]
T_D_HRI.columns = ['e2','k','k1']
# when k is another handoff spots
HRI_HRI = pd.merge(HRI, HRI, how='cross')
HRI_HRI['Keep']=HRI_HRI.apply(lambda x: T_NodeSeqFeasibility(x), axis = 1)
HRI_HRI = HRI_HRI[HRI_HRI['Keep']].reset_index(drop=True)[['Node_x','Node_y']]
T_HRI_HRI = pd.merge(T,HRI_HRI, how='cross')
T_HRI_HRI.columns = ['e2','k','k1']

# get the final sets and dict
Y_T = pd.concat([T_H_HRI, T_HRI_D, T_D_HRI, T_D_H, T_HRI_HRI], ignore_index=True)
Y_T['Cell'] = 1
DYt, _ = multidict(Y_T.set_index(['e2', 'k', 'k1']).to_dict()['Cell'])

# create the decision variable
Yt = model.addVars(DYt, vtype=GRB.BINARY, name = 'Yt')

In [194]:
Y_T

,e2,k,k1,Cell
0,1,MyHome_T1,MyPickNode1_I1,1
1,1,MyHome_T1,MyPickNode2_I1,1
2,1,MyHome_T1,MyPickNode3_I1,1
3,1,MyHome_T1,MyPickNode4_I1,1
4,1,MyHome_T1,MyPickNode5_I1,1
...,...,...,...,...
89125,3,MyCoopNode12_I5,MyCoopNode8_I4,1
89126,3,MyCoopNode12_I5,MyCoopNode9_I4,1
89127,3,MyCoopNode12_I5,MyCoopNode10_I4,1
89128,3,MyCoopNode12_I5,MyCoopNode11_I4,1


### Picker arrival time decision variables -- t[e1, k]

In [195]:
# All nodes that a picker may visit (exclude home node)
P_AllNodes = pd.concat([RI, HRI[['Node']]], ignore_index=True)
tp_df = pd.merge(pickers[['PickerID','Capacity']],P_AllNodes, how='cross' )
# create dicts for decision variable
Dtp, _ = multidict(tp_df.set_index(['PickerID','Node']).to_dict()['Capacity'])
# create the decision variable
tp = model.addVars(Dtp, vtype=GRB.CONTINUOUS, lb=0, name = 'tp')

### Picker payload at node decision variables-- S[e1,k]

In [196]:
# create the decision variable
Sp = model.addVars(Dtp, vtype=GRB.INTEGER, ub = tp_df['Capacity'].tolist(), name = 'Sp')

### Transporter arrival time decision variables -- t[e2, k]

In [197]:
# All nodes that a transporter may visit (exclude home node)
NewD = D.copy()
NewD.columns = ['Node']
T_AllNodes = pd.concat([NewD, HRI[['Node']]], ignore_index=True)
tt_df = pd.merge(transporters[['TransporterID','Capacity']],T_AllNodes, how='cross')
# create dicts for decision variable
Dtt, _ = multidict(tt_df.set_index(['TransporterID','Node']).to_dict()['Capacity'])
# create the decision variable
tt = model.addVars(Dtt, vtype=GRB.CONTINUOUS, lb=0, name = 'tt')

In [198]:
Dtt

<gurobi.tuplelist (570 tuples, 2 values each):
 ( 1 , DepotRep_1      )
 ( 1 , DepotRep_2      )
 ( 1 , DepotRep_3      )
 ( 1 , DepotRep_4      )
 ( 1 , DepotRep_5      )
 ( 1 , DepotRep_6      )
 ( 1 , DepotRep_7      )
 ( 1 , DepotRep_8      )
 ( 1 , DepotRep_9      )
 ( 1 , DepotRep_10     )
 ( 1 , MyPickNode1_I1  )
 ( 1 , MyPickNode2_I1  )
 ( 1 , MyPickNode3_I1  )
 ( 1 , MyPickNode4_I1  )
 ( 1 , MyPickNode5_I1  )
 ( 1 , MyPickNode6_I1  )
 ( 1 , MyPickNode7_I1  )
 ( 1 , MyPickNode8_I1  )
 ( 1 , MyPickNode9_I1  )
 ( 1 , MyPickNode10_I1 )
 ( 1 , MyPickNode11_I1 )
 ( 1 , MyPickNode12_I1 )
 ( 1 , MyPickNode13_I1 )
 ( 1 , MyPickNode14_I1 )
 ( 1 , MyPickNode15_I1 )
 ( 1 , MyPickNode16_I1 )
 ( 1 , MyPickNode17_I1 )
 ( 1 , MyPickNode18_I1 )
 ( 1 , MyPickNode19_I1 )
 ( 1 , MyPickNode20_I1 )
 ( 1 , MyPickNode21_I1 )
 ( 1 , MyPickNode22_I1 )
 ( 1 , MyPickNode23_I1 )
 ( 1 , MyPickNode24_I1 )
 ( 1 , MyCoopNode1_I1  )
 ( 1 , MyCoopNode2_I1  )
 ( 1 , MyCoopNode3_I1  )
 ( 1 , MyCoopNode4_I1  )
 ( 

### Transporter payload at node decision variables -- S[e2,k]

In [199]:
# create the decision variable
St = model.addVars(Dtt, vtype=GRB.INTEGER, ub = tt_df['Capacity'].tolist(), name = 'St')

### Cooperation flag decision variables -- H[i]

In [200]:
# new all item df
NewI = I.copy()
NewI['Cell'] = 1
# create dicts for decision variable
DH, _ = multidict(NewI.set_index(['ItemID']).to_dict()['Cell'])
# create the decision variable
H = model.addVars(DH, vtype=GRB.BINARY, name = 'Handoff')

### Some useful lists and dicts for building constraints

In [201]:
# Picking Locations list
PLsL = RI['Node'].tolist()
# all handoff replicas list
HRsL = HRI['Node'].tolist()
# pickers list
PsL = pickers['PickerID'].tolist()
# pikcer home list
PHL = Hp['k'].tolist()
# transporter list
TsL = transporters['TransporterID'].tolist()
# transporter home list
THL = Ht['k1'].tolist()
# all items list (include virtual items)
IsL = lineItem['ItemID'].tolist()
IsL.extend([0,-1])
# all depot replicas list
DpsL = D['Reps'].tolist()
# bigM for all time related constraints
BigMtime = 10000

In [202]:
DpsL+HRsL

['DepotRep_1',
 'DepotRep_2',
 'DepotRep_3',
 'DepotRep_4',
 'DepotRep_5',
 'DepotRep_6',
 'DepotRep_7',
 'DepotRep_8',
 'DepotRep_9',
 'DepotRep_10',
 'MyPickNode1_I1',
 'MyPickNode2_I1',
 'MyPickNode3_I1',
 'MyPickNode4_I1',
 'MyPickNode5_I1',
 'MyPickNode6_I1',
 'MyPickNode7_I1',
 'MyPickNode8_I1',
 'MyPickNode9_I1',
 'MyPickNode10_I1',
 'MyPickNode11_I1',
 'MyPickNode12_I1',
 'MyPickNode13_I1',
 'MyPickNode14_I1',
 'MyPickNode15_I1',
 'MyPickNode16_I1',
 'MyPickNode17_I1',
 'MyPickNode18_I1',
 'MyPickNode19_I1',
 'MyPickNode20_I1',
 'MyPickNode21_I1',
 'MyPickNode22_I1',
 'MyPickNode23_I1',
 'MyPickNode24_I1',
 'MyCoopNode1_I1',
 'MyCoopNode2_I1',
 'MyCoopNode3_I1',
 'MyCoopNode4_I1',
 'MyCoopNode5_I1',
 'MyCoopNode6_I1',
 'MyCoopNode7_I1',
 'MyCoopNode8_I1',
 'MyCoopNode9_I1',
 'MyCoopNode10_I1',
 'MyCoopNode11_I1',
 'MyCoopNode12_I1',
 'MyPickNode1_I2',
 'MyPickNode2_I2',
 'MyPickNode3_I2',
 'MyPickNode4_I2',
 'MyPickNode5_I2',
 'MyPickNode6_I2',
 'MyPickNode7_I2',
 'MyPickNode8_

## Constraints

In [203]:
# constraint No.1
for e1, k in Dtp:
    model.addConstr(m >=tp[e1,k])

In [204]:
# constraint No.7
# each item will only be picked once
# each item
for i in range(1,NumItems+1):
    # the next item list
    jlist = IsL.copy()
    jlist.remove(i)
    jlist.remove(0)
    # add constraints
    model.addConstr(quicksum(quicksum(X[e1,i,j] for j in jlist) for e1 in PsL) == 1)

# each item
for j in range(1,NumItems+1):
    # the previous item list
    ilist = IsL.copy()
    ilist.remove(j)
    ilist.remove(-1)        
    # add constraints
    model.addConstr(quicksum(quicksum(X[e1,i,j] for i in ilist) for e1 in PsL) == 1) 

In [205]:
# constraint No.18
# each picker will start from picking item 0 and end from picking item -1
for e1 in PsL:
    model.addConstr(quicksum(X[e1,i,-1] for i in range(1,NumItems+1)) == 1)
    model.addConstr(quicksum(X[e1,0,j] for j in range(1,NumItems+1)) == 1)

In [206]:
# constraint No.2a
# each picking location will only be visited by one picker
# each picking location
ConstrNo = 1
for ri in PLsL:
    # the previous possible nodes list of list
    klist = []
    # each picker
    for e1 in PsL:
        klist.append(P_HHRI_RI['k'][(P_HHRI_RI['e1'] == e1) & (P_HHRI_RI['k1'] == ri)].tolist())
    # add constraints
    name = 'C'+str(ConstrNo)+'_'+str(PLsL.index(ri))
    model.addConstr(quicksum(quicksum(Yp[e1,k,ri] for k in klist[e1-1]) for e1 in PsL) ==1, name)

In [207]:
# constraint No.2b
# after visiting a picking location, the picker will move to one of the handoff replicas of that item
# each picking location
for ri in PLsL:
    # each picker
    for e1 in PsL:
        # the possible next node list
        k1list = P_RI_HRI['k1'][(P_RI_HRI['e1'] == e1) & (P_RI_HRI['k'] == ri)].tolist()
    # add constraints
    model.addConstr(quicksum(quicksum(Yp[e1,ri,k1] for k1 in k1list) for e1 in PsL) ==1)

In [208]:
# constraint No.2c
# a picker will be back to its home node after picking all needed items
# each picker
for e1 in PsL:
    # home node for the picker
    k1 =  PHL[e1-1]
    # add constraints
    model.addConstr(quicksum(Yp[e1,k,k1] for k in HRsL) == 1)

In [209]:
# constraint No.2d
# a picker will start from its home node
# each picker
for e1 in PsL:
    # home node for the picker
    k = PHL[e1-1]
    # add constraints
    model.addConstr(quicksum(Yp[e1,k,k1] for k1 in PLsL) == 1)

In [210]:
# constraints No.3
# each handoff node
k1list = HRsL + DpsL
for k1 in k1list:
    e2klist = Y_T[Y_T['k1'] == k1][['e2','k']].values.tolist()
#     e2k2list = Y_T[Y_T['k'] == k1][['e2','k1']].values.tolist()
    # constraint 3.a
    # each handoff node and depot replica will be visited by no more than one transporter
    model.addConstr(quicksum(Yt[e2,k,k1] for e2,k in e2klist) <= 1)

In [211]:
# constraints No.3
# each transporter
for e2 in TsL:
    # each handoff node
    k1list = HRsL + DpsL
    for k1 in k1list:
        klist = Y_T[(Y_T['k1'] == k1) & (Y_T['e2'] == e2)]['k'].tolist()
        k2list = Y_T[(Y_T['k'] == k1) & (Y_T['e2'] == e2)]['k1'].tolist()
        # 3.b when a transporter enter a handoff node or a depot replica, constarint ensures that it will leave that node
        model.addConstr(quicksum(Yt[e2,k,k1] for k in klist) == quicksum(Yt[e2,k1,k2] for k2 in k2list))

In [212]:
# contraint No.3
for e2 in TsL:
    k1 = THL[e2-1]
    # constraints 3.c  a transporter will be back to its home node
    model.addConstr(quicksum(Yt[e2,k,k1] for k in DpsL) == 1)

for e2 in TsL:
    k = THL[e2-1]
    # constraint 3.d  a transporter will start from its home node
    model.addConstr(quicksum(Yt[e2,k,k1] for k1 in HRsL) == 1)

In [213]:
# # constraints No.3
# for k1 in DpsL:
#     e2klist = Y_T[Y_T['k1'] == k1][['e2','k']].values.tolist()
#     e2k2list = Y_T[Y_T['k'] == k1][['e2','k1']].values.tolist() 
#     # constraint 3.e
#     # each depot replica can only be visited once
#     model.addConstr(quicksum(Yt[e2,k,k1] for e2,k in e2klist) <= 1)
#     # constraint 3.f
#     # when a transporter enter a depot replica, constarint ensures that it will leave that node
#     model.addConstr(quicksum(Yt[e2,k,k1] for e2,k in e2klist) == quicksum(Yt[e2,k1,k2] for e2, k2 in e2k2list))

In [214]:
# constraints No.3
# each transporter
# each item
for i in range(1,NumItems+1):
    k1list = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    expr = LinExpr()
    for e2 in TsL:
        for k1 in k1list:
            klist = Y_T[(Y_T['k1'] == k1) & (Y_T['e2'] == e2)]['k'].tolist()
            expr.add(quicksum(Yt[e2,k,k1] for k in klist))
    # 3.e ensure the relationshiop between Yt and H
    model.addConstr(expr == H[i])

In [215]:
# constraint No.3
BigM = 2
# each item
for i in range(1,NumItems+1):
    Ri = PLsL[i-1]
    k1list = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    for k1 in k1list:
        for e2 in TsL:
            klist = Y_T[(Y_T['k1'] == k1) & (Y_T['e2'] == e2)]['k'].tolist()
            for e1 in PsL:
                # 3.f the assigned picker and transporter will meet at a handoff node of an item when Hi = 1
                model.addConstr(Yp[e1,Ri,k1] <= quicksum(Yt[e2,k,k1] for k in klist) + BigM*(2-H[i]-a[e1,e2,i]))
                model.addConstr(quicksum(Yt[e2,k,k1] for k in klist) <= Yp[e1,Ri,k1] + BigM*(2-H[i]-a[e1,e2,i]))

In [216]:
# constaint No.4
# each item can only be retrieved by one picker and be placed on one transporter
for i in range(1,NumItems+1):
    model.addConstr(quicksum(quicksum(a[e1,e2,i] for e2 in TsL) for e1 in PsL) == 1)

In [217]:
# constraints No.6
# set the node sequence constraint for pickers after picking an item
# each picker
for e1 in PsL:
    # each item
    for i in range(1,NumItems+1):
        Ri_star = PLsL[i-1][:PLsL[i-1].find('_')] + '_I' + str(i)
        model.addConstr(Yp[e1,PLsL[i-1], Ri_star] >= 1- H[i])

In [218]:
# constraint No.20
# an item can only picked by one picker 
#each item
for j in range(1,NumItems+1):
    ilist = list(range(0,NumItems+1))
    ilist.remove(j)
    llist = list(range(1,NumItems+1))
    llist.remove(j)
    llist.append(-1)
    # each picker
    for e1 in PsL:
        model.addConstr(quicksum(X[e1,i,j] for i in ilist) == quicksum(X[e1,j,l] for l in llist))

In [219]:
# constraint No.8
# to set appropriate node sequence of picker based on X[e1,i,j]
# each item i
for i in range(1,NumItems+1):
    # the next item
    jlist = list(range(1,NumItems+1))
    jlist.remove(i)
    kk1list = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    # each picker
    for e1 in PsL:
        HNe1 = PHL[e1-1]
        # the next item
        for j in jlist:      
            # add constraints
            model.addConstr(2*X[e1,i,j] <= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,PLsL[j-1]] for kk1 in kk1list))
            model.addConstr(X[e1,i,j]+1 >= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,PLsL[j-1]] for kk1 in kk1list))
        model.addConstr(2*X[e1,i,-1] <= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,HNe1] for kk1 in kk1list))
        model.addConstr(X[e1,i,-1]+1 >= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,HNe1] for kk1 in kk1list))
        model.addConstr(2*X[e1,0,i] <= quicksum(Yp[e1,PLsL[i-1],kk1] for kk1 in kk1list) + Yp[e1,HNe1,PLsL[i-1]])
        model.addConstr(X[e1,0,i]+1 >= quicksum(Yp[e1,PLsL[i-1],kk1] for kk1 in kk1list) + Yp[e1,HNe1,PLsL[i-1]])

In [220]:
# constraint No.9
# the time picker travel from picking node to handoff node
PKT = 30
# each picker
for e1 in PsL:
    # each item
    for i in range(1,NumItems+1):
        # the handoff replicas of item i
        klist = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
        Ri = PLsL[i-1]
        # each handoff node
        for k in klist:
            model.addConstr(tp[e1,k] >= tp[e1,Ri] + PKT + Lp[Ri,k]/Vp[e1] - BigMtime*(1-Yp[e1,Ri,k]))

In [221]:
# constraint No.10
# the time picker travel from handoff node to picking node
# each picker
for e1 in PsL:
    # each item
    for i in range(1,NumItems+1): 
        jlist = list(range(1,NumItems+1))
        jlist.remove(i)
        Ri_star = PLsL[i-1][:PLsL[i-1].find('_')] + '_I' + str(i)
        # the handoff replicas of item i
        klist = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
        # the next item
        for j in jlist:
            Rj = PLsL[j-1]
            # 10.a when H[i] = 0
            model.addConstr(tp[e1,Rj] >= tp[e1,Ri_star] + Lp[Ri_star,Rj]/Vp[e1] - BigMtime*(1-X[e1,i,j]+H[i]))
            # each handoff node of item i
            for k in klist:
                # 10.b when H[i] = 1
                model.addConstr(tp[e1,Rj] >=tp[e1,k] + Lp[k,Rj]/Vp[e1] + PLT[e1]*Sp[e1,k] - BigMtime*(3-X[e1,i,j]-Yp[e1,k,Rj]-H[i]))

In [222]:
# constraint No.11
# The time picker travel from home node to picking node
# each picker
for e1 in PsL:
    HNe1 = PHL[e1-1]
    # each item
    for i in range(1,NumItems+1):
        Ri = PLsL[i-1]
        model.addConstr(tp[e1,Ri] >= Lp[HNe1,Ri]/Vp[e1] - BigMtime*(1-Yp[e1,HNe1,Ri]))
#         model.addConstr(tp[e1,Ri] <= Lp[HNe1,Ri]/Vp[e1] + BigMtime*(1-Yp[e1,HNe1,Ri]))

In [223]:
# constraints No.12
# the constraints about picker's payload at picking node
# each picker
for e1 in PsL:
    # each item
    for i in range(1,NumItems+1):
        jlist = list(range(1,NumItems+1))
        jlist.remove(i)
        # the next item
        for j in jlist:
            model.addConstr(Sp[e1,PLsL[j-1]] >= -Cp[e1]*(2-X[e1,i,j]-H[i]))
            model.addConstr(Sp[e1,PLsL[j-1]] <= Cp[e1]*(2-X[e1,i,j]-H[i]))            
            Ri_star = PLsL[i-1][:PLsL[i-1].find('_')] + '_I' + str(i)
            model.addConstr(Sp[e1,PLsL[j-1]] >= -Cp[e1]*(1-X[e1,i,j]+H[i]) + Sp[e1,Ri_star])
            model.addConstr(Sp[e1,PLsL[j-1]] <= Cp[e1]*(1-X[e1,i,j]+H[i]) + Sp[e1,Ri_star])

In [224]:
# constraints No.13
# each picker
for e1 in PsL:
    HNe1 = PHL[e1-1]
    # each item:
    for i in range(1,NumItems+1):
        Ri = PLsL[i-1]
        # 13.a the constraints ensure that each picker has zero payload when reach the its first picking node 
        model.addConstr(Sp[e1,Ri] >= -Cp[e1]*(1-Yp[e1,HNe1,Ri]))
        model.addConstr(Sp[e1,Ri] <=  Cp[e1]*(1-Yp[e1,HNe1,Ri]))
        # 13.b the constraints ensure that the payload of unrelated first picking location is zero for a picker
        elist = PsL.copy()
        elist.remove(e1)
        expr = LinExpr(1)
        for e in elist:
            HNe = PHL[e-1]
            expr.add(-Yp[e,HNe,Ri])
        model.addConstr(Sp[e1,Ri] >= -Cp[e1]*expr)
        model.addConstr(Sp[e1,Ri] <=  Cp[e1]*expr)


In [225]:
# constraint No.14
# the constraints about picker's payload at handoff node
# each picker
for e1 in PsL:
    # each item
    for i in range(1,NumItems+1):
        # the handoff replicas of item i
        klist = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
        for k in klist:
            model.addConstr(Sp[e1,k] >= -Cp[e1]*(1-Yp[e1,PLsL[i-1],k]) + Sp[e1,PLsL[i-1]] + 1)
            model.addConstr(Sp[e1,k] <= Cp[e1]*(1-Yp[e1,PLsL[i-1],k]) + Sp[e1,PLsL[i-1]] + 1)
            model.addConstr(Sp[e1,k] >= -Cp[e1]*(Yp[e1,PLsL[i-1],k]))
            model.addConstr(Sp[e1,k] <= Cp[e1]*(Yp[e1,PLsL[i-1],k]))

In [226]:
# constraint No.19
# to set appropriate node sequence of picker based on X[e1,i,j]
BigM = 2
# each item i
for i in range(1,NumItems+1):
    jlist = list(range(1,NumItems+1))
    jlist.remove(i)
    kk1list = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    # each picker       
    for e1 in PsL:
        HNe1 = 'MyHome_P' + str(e1)
        # add constraints 19.b
        # each node of HRi
        for kk1 in kk1list:
            model.addConstr(Yp[e1,PLsL[i-1],kk1] <= Yp[e1,kk1,HNe1] + BigM*(1-X[e1,i,-1]))
            model.addConstr(Yp[e1,kk1,HNe1] <= Yp[e1,PLsL[i-1],kk1] + BigM*(1-X[e1,i,-1]))
        # the next item
        for j in jlist:
            # each node of HRi
            for kk1 in kk1list:
            # add constraints 19.a
                model.addConstr(Yp[e1,PLsL[i-1],kk1] <= Yp[e1,kk1,PLsL[j-1]] + BigM*(1-X[e1,i,j]))
                model.addConstr(Yp[e1,kk1,PLsL[j-1]] <= Yp[e1,PLsL[i-1],kk1] + BigM*(1-X[e1,i,j]))  

In [227]:
obj = m
model.setObjective(obj, GRB.MINIMIZE)
# solve model
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10564 rows, 93276 columns and 726375 nonzeros
Model fingerprint: 0xc7220f84
Variable types: 941 continuous, 92335 integer (91395 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 3e+04]
Presolve removed 605 rows and 1164 columns
Presolve time: 1.03s

Explored 0 nodes (0 simplex iterations) in 1.12 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [177]:
# #display solution

# print("Obj: {:.5f}".format(model.objVal))

# model.printAttr('Xn')
model.setParam('SolutionNumber', 0)
model.printAttr('Xn')


    Variable           Xn 
-------------------------
           m      260.005 
    A[1,3,1]            1 
    A[2,1,2]            1 
    A[1,3,3]            1 
    A[2,1,4]            1 
    A[1,3,5]            1 
    X[1,5,4]            1 
    X[1,0,5]            1 
   X[1,4,-1]            1 
    X[2,1,2]            1 
    X[2,3,1]            1 
    X[2,0,3]            1 
   X[2,2,-1]            1 
Yp[1,MyPickNode18_PL4,MyCoopNode9_I4]            1 
Yp[1,MyPickNode18_PL5,MyPickNode18_I5]            1 
Yp[2,MyPickNode3_PL1,MyPickNode4_I1]            1 
Yp[2,MyPickNode5_PL2,MyPickNode5_I2]            1 
Yp[2,MyPickNode4_PL3,MyPickNode4_I3]            1 
Yp[1,MyPickNode18_I5,MyPickNode18_PL4]            1 
Yp[2,MyPickNode4_I3,MyPickNode3_PL1]            1 
Yp[2,MyPickNode4_I1,MyPickNode5_PL2]            1 
Yp[1,MyHome_P1,MyPickNode18_PL5]            1 
Yp[2,MyHome_P2,MyPickNode4_PL3]            1 
Yp[1,MyCoopNode9_I4,MyHome_P1]            1 
Yp[2,MyPickNode5_I2,MyHome_P2]            1 

In [178]:
count = 0
for v in model.getVars():
    if 'Handoff' in v.VarName and v.x==1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(keys)
print(count)

['1']
['2']
['3']
['4']
['5']
5


In [181]:
count = 0
for v in model.getVars():
    if 'X' in v.VarName and v.x==1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(keys)
print(count)

['1', '5', '4']
['1', '0', '5']
['1', '4', '-1']
['2', '1', '2']
['2', '3', '1']
['2', '0', '3']
['2', '2', '-1']
7


In [179]:
count = 0
for v in model.getVars():
    if 'Yp' in v.VarName and v.x==1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(keys)
print(count)

['1', 'MyPickNode18_PL4', 'MyCoopNode9_I4']
['1', 'MyPickNode18_PL5', 'MyPickNode18_I5']
['2', 'MyPickNode3_PL1', 'MyPickNode4_I1']
['2', 'MyPickNode5_PL2', 'MyPickNode5_I2']
['2', 'MyPickNode4_PL3', 'MyPickNode4_I3']
['1', 'MyPickNode18_I5', 'MyPickNode18_PL4']
['2', 'MyPickNode4_I3', 'MyPickNode3_PL1']
['2', 'MyPickNode4_I1', 'MyPickNode5_PL2']
['1', 'MyHome_P1', 'MyPickNode18_PL5']
['2', 'MyHome_P2', 'MyPickNode4_PL3']
['1', 'MyCoopNode9_I4', 'MyHome_P1']
['2', 'MyPickNode5_I2', 'MyHome_P2']
12


In [180]:
count = 0
for v in model.getVars():
    if 'Yt' in v.VarName and v.x==1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(keys)
print(count)

['1', 'MyHome_T1', 'MyPickNode5_I2']
['2', 'MyHome_T2', 'MyPickNode1_I3']
['3', 'MyHome_T3', 'MyPickNode5_I4']
['1', 'MyPickNode5_I2', 'DepotRep_1']
['2', 'MyPickNode1_I1', 'DepotRep_3']
['2', 'MyPickNode1_I3', 'DepotRep_2']
['3', 'MyPickNode18_I5', 'DepotRep_7']
['2', 'DepotRep_3', 'MyPickNode1_I1']
['1', 'DepotRep_1', 'MyHome_T1']
['2', 'DepotRep_2', 'MyHome_T2']
['3', 'DepotRep_7', 'MyHome_T3']
['3', 'MyPickNode5_I4', 'MyPickNode18_I5']
12


In [723]:
count = 0
for v in model.getVars():
    if 'Sp' in v.VarName and v.x>0:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(f'{keys}, {v.x}')
print(count)

['1', 'MyCoopNode11_I1'], 1.0
['1', 'MyPickNode22_I2'], 1.0
['1', 'MyPickNode22_I3'], 1.0
['1', 'MyPickNode15_I6'], 1.0
['1', 'MyPickNode13_I7'], 1.0
['1', 'MyCoopNode11_I11'], 1.0
['1', 'MyPickNode13_I12'], 1.0
['1', 'MyPickNode14_I16'], 1.0
['2', 'MyPickNode8_I4'], 1.0
['2', 'MyPickNode8_I5'], 1.0
['2', 'MyPickNode12_I8'], 1.0
['2', 'MyPickNode12_I9'], 1.0
['2', 'MyPickNode15_I10'], 1.0
['2', 'MyPickNode12_I13'], 1.0
['2', 'MyPickNode12_I14'], 1.0
['2', 'MyPickNode3_I15'], 1.0
16


In [724]:
count = 0
for v in model.getVars():
    if 'tp' in v.VarName and v.x>0:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(f'{keys}, {v.x}')
print(count)

['1', 'MyPickNode22_PL1'], 710.0292512786273
['1', 'MyPickNode22_PL2'], 560.0292512786277
['1', 'MyPickNode22_PL3'], 635.0292512786277
['1', 'MyPickNode13_PL6'], 325.0099024020567
['1', 'MyPickNode13_PL7'], 175.0099024020567
['1', 'MyPickNode15_PL11'], 430.0083846122691
['1', 'MyPickNode13_PL12'], 250.0099024020567
['1', 'MyPickNode12_PL13'], 745.0297603779636
['1', 'MyPickNode3_PL16'], 30.009904136997648
['1', 'MyCoopNode11_I1'], 745.0297603779636
['1', 'MyPickNode22_I2'], 590.0292512786277
['1', 'MyPickNode22_I3'], 665.0292512786277
['1', 'MyPickNode1_I5'], 745.0297603779636
['1', 'MyPickNode2_I5'], 745.0297603779636
['1', 'MyPickNode3_I5'], 745.0297603779636
['1', 'MyPickNode4_I5'], 745.0297603779636
['1', 'MyPickNode5_I5'], 745.0297603779636
['1', 'MyPickNode8_I5'], 745.0297603779636
['1', 'MyPickNode9_I5'], 745.0297603779636
['1', 'MyPickNode11_I5'], 745.0297603779636
['1', 'MyPickNode13_I5'], 745.0297603779636
['1', 'MyPickNode15_I5'], 745.0297603779636
['1', 'MyPickNode16_I5'], 

In [725]:
model.getVars()

[<gurobi.Var m (value 745.0297603779636)>,
 <gurobi.Var A[1,1,1] (value 1.0)>,
 <gurobi.Var A[1,2,1] (value 0.0)>,
 <gurobi.Var A[1,3,1] (value 0.0)>,
 <gurobi.Var A[2,1,1] (value 0.0)>,
 <gurobi.Var A[2,2,1] (value 0.0)>,
 <gurobi.Var A[2,3,1] (value 0.0)>,
 <gurobi.Var A[1,1,2] (value 1.0)>,
 <gurobi.Var A[1,2,2] (value 0.0)>,
 <gurobi.Var A[1,3,2] (value 0.0)>,
 <gurobi.Var A[2,1,2] (value 0.0)>,
 <gurobi.Var A[2,2,2] (value 0.0)>,
 <gurobi.Var A[2,3,2] (value 0.0)>,
 <gurobi.Var A[1,1,3] (value 1.0)>,
 <gurobi.Var A[1,2,3] (value 0.0)>,
 <gurobi.Var A[1,3,3] (value 0.0)>,
 <gurobi.Var A[2,1,3] (value 0.0)>,
 <gurobi.Var A[2,2,3] (value 0.0)>,
 <gurobi.Var A[2,3,3] (value 0.0)>,
 <gurobi.Var A[1,1,4] (value 1.0)>,
 <gurobi.Var A[1,2,4] (value 0.0)>,
 <gurobi.Var A[1,3,4] (value 0.0)>,
 <gurobi.Var A[2,1,4] (value 0.0)>,
 <gurobi.Var A[2,2,4] (value 0.0)>,
 <gurobi.Var A[2,3,4] (value 0.0)>,
 <gurobi.Var A[1,1,5] (value 1.0)>,
 <gurobi.Var A[1,2,5] (value 0.0)>,
 <gurobi.Var A[1,3,5]

In [174]:
cc

[2]

In [168]:
PsL

[1, 2]